# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [3]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using ManifoldDiff
using FiniteDiff
using FiniteDifferences

In [183]:
include("../../../src/decompositions/naive_SVD.jl")
include("../../../src/utils/curvature_corrected_loss.jl")
# include("../../../src/utils/gradient_curvature_corrected_loss.jl")
include("../../../src/decompositions/curvature_corrected_low_rank_approximation.jl")

curvature_corrected_low_rank_approximation (generic function with 2 methods)

### Load data and construct manifold ###

In [172]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 100  # 100


100

In [173]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[1]

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


In [163]:
# Export slice image
asymptote_export_SPD("results/artificial1D_orig.asy", data=data);

### Construct low rank approximation ###

In [174]:
q = mean(M, data)
# q = 1. * Matrix(I, 3, 3)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [165]:
# TOY -- test
# compute curvature corrected loss
Ξ = 1/2 .* log_q_data
i = 1
loss = curvature_corrected_loss(M, e, data, Ξ[i], i)

0.1908537095052726

In [186]:
# TOY -- test
# gradient_curvature_corrected_loss()
curvature_corrected_low_rank_approximation(M, e, data, 1; stepsize=1/100000);

Initial  F(x): 0.97109675402 | 
# 1     change: 0.003567717 |  F(x): 0.11684527545 | 
# 2     change: 0.001221225 |  F(x): 0.01673893463 | 
# 3     change: 0.000418290 |  F(x): 0.00499234909 | 
# 4     change: 0.000143394 |  F(x): 0.00361152157 | 
# 5     change: 0.000049216 |  F(x): 0.00344879542 | 
# 6     change: 0.000016920 |  F(x): 0.00342955117 | 
# 7     change: 0.000005831 |  F(x): 0.00342726416 | 
# 8     change: 0.000002016 |  F(x): 0.00342699051 | 
# 9     change: 0.000000700 |  F(x): 0.00342695743 | 
# 10    change: 0.000000245 |  F(x): 0.00342695336 | 
# 11    change: 0.000000088 |  F(x): 0.00342695282 | 
# 12    change: 0.000000032 |  F(x): 0.00342695271 | 
# 13    change: 0.000000017 |  F(x): 0.00342695267 | 
# 14    change: 0.000000019 |  F(x): 0.00342695263 | 
# 15    change: 0.000000001 |  F(x): 0.00342695259 | 
# 16    change: 0.000000019 |  F(x): 0.00342695255 | 
# 17    change: 0.000000019 |  F(x): 0.00342695251 | 
# 18    change: 0.000000019 |  F(x): 0.00342695247

In [23]:
N_R_q, N_U = naive_SVD(M,q, data);
CR_R_q, CR_U = curvature_reweighed_SVD(M,q, data);

In [24]:
ref_distance = sqrt(sum(distance.(Ref(M), Ref(q), data).^2))

N_tangent_distances_r = zeros(d)
N_distances_r = zeros(d)
CR_tangent_distances_r = zeros(d)
CR_distances_r = zeros(d)

for rank in 1:d
    N_log_q_data_r = Symmetric.([sum([N_R_q[i] * N_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    CR_log_q_data_r = Symmetric.([sum([CR_R_q[i] * CR_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    # expoentiate back
    N_data_r = exp.(Ref(M), Ref(q), N_log_q_data_r)
    CR_data_r = exp.(Ref(M), Ref(q), CR_log_q_data_r)


    # compute relative tangent space error
    N_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - N_log_q_data_r).^2)) / ref_distance
    CR_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - CR_log_q_data_r).^2)) / ref_distance


    # compute relative manifold error
    N_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, N_data_r).^2)) / ref_distance
    CR_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, CR_data_r).^2)) / ref_distance
end

In [25]:
plot(1:d, [N_tangent_distances_r, CR_tangent_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,d))
savefig("results/artificial1D_tangent_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_tangent_rank.png"

In [26]:
plot(1:d, [N_distances_r, CR_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,d))
savefig("results/artificial1D_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_rank.png"